In [ ]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np
import re, string, nltk
from collections import defaultdict
from nltk.corpus import stopwords

# Object-oriented Programming

In order for this analysis to scale and add the text from upcoming debates, I believe having all of a candidates data in a single area will make adding new data easier. So first, I'll make a Speaker class, read in the first 2 nights of debate, and populate each candidate's class.

In [16]:
NUMBER_OF_TRANSCRIPTS = 4
class Speaker:
    def __init__(self, name):
        self.count = 0
        self.name = name #name of candidate
        self.text = [[] for _ in range(NUMBER_OF_TRANSCRIPTS)] # this will allow us to know if someone did not speak on a particular night
        
    def addConvo(self, convo, night_number):
        self.text[night_number].append(convo)
        self.cont += 1
        
    
    def getConvos(self):
        return self.text
    
    def getConvosOfNightK(self, k):
        return self.text[k]
    
    
    def printConvosNightK(self, k=0): #default is to print all of the convos
        if k:
            for convo in self.text[k]:
                    print(convo)
        else:
            for night in range(NUMBER_OF_TRANSCRIPTS):
                for convo in self.text[night]:
                    print(convo)
            
    def contibutionsNightK(self, k):
        return len(self.text[k])
    
    def totalContributions(self):
        return self.count
        

Unfortunately, I could not find the transcripts of round 2 of the debates through Time magazine. Bloomberg has each night [here](https://www.bloomberg.com/graphics/democratic-presidential-debates-key-issues/#span-67). So I will be using these transcripts for this analysis. Their site has a wonderful visualization of the topics of discussion throughout the debates. I am going to analyzed the rhetoric and sentiment used throughout the debates.

In [10]:
def readTranscripts(fileName):
    f = open(fileName)
    lines = f.readlines()
    f.close()
    return ''.join(lines).split('\n')

def isSpeaker(line):
    return line.upper() == line

def isModerator(lines, i):
    return i + 1 < len(lines) and isSpeaker(lines[i]) and lines[i + 1] == 'MODERATOR'

def addConvoToSpeaker(mapping, name, convo, nightNumber):
    if name not in mapping:
        mapping[name] = Speaker(name)
    speaker = mapping[name]
    speaker.addConvo(convo, nightNumber)
    

def loadNightK(k, mods, cands):
    transcript = readTranscripts('./transcripts/bloomberg_night_' + str(k + 1) + '.txt')
    i = 0
    while i + 1 < len(transcript):
        name = transcript[i]
        if isModerator(transcript, i):
            addConvoToSpeaker(mods, name, transcript[i + 2], k)        
            i += 4
        else:
            addConvoToSpeaker(cands, name, transcript[i + 1], k) 
            i += 3 

In [11]:
moderators = {}
candidates = {}
for k in range(NUMBER_OF_TRANSCRIPTS):
    loadNightK(k, moderators, candidates)

Great, now we have loaded all the 

In [15]:
for cand in candidates:
    if '(?)' in cand:
        print(cand + ': ', candidates[cand].getConvos())

SANDERS (?):  [[], ['I just...'], [], []]
WILLIAMSON (?):  [[], ['Another thing, I...'], [], []]
SWALWELL (?):  [[], ["Here's the solution. Pass the torch. Pass the torch to the generation that's going to feel the effects of climate change."], [], []]


In [ ]:
candidates.keys()

In [ ]:
def analyzeCandidates(lst, night1=False):
    """
    input: List(str)= the conversations of each night, where each speaker has a colon in the last place
    output: dictionary mapping speakers to a list of their inputs during the night.
    """
    
    def switchDeBlasio(listOfWords):
        i = 0
        while i < len(listOfWords):
            if listOfWords[i] == "DE" and listOfWords[i + 1] == "BLASIO:":
                listOfWords.pop(i)
                listOfWords[i] = "DEBLASIO:"
            i += 1
        return listOfWords
    
    speakers = findSpeakers(lst, night1)
    speakersToText = {}
    for s in speakers:
        speakersToText[s] = Speaker(s)
        #print('\t', s)
    
    wordByWord = switchDeBlasio(''.join(lst).split())
    
    start = 0
    end = 1
    while end < len(wordByWord):
        speaker = wordByWord[start][:-1]
        mightSpeak = wordByWord[end]
        if isSpeaker(mightSpeak, speakers):
            newConvo = ' '.join(wordByWord[start + 1:end])
            speakersToText[speaker].addConvo(newConvo)
            start = end
        end += 1
    return speakersToText   

def findSpeakers(lst, deBlasio=False):
    speakers = set()
    for s in lst:
        if ":" in s:
            idx = s.index(":")
            if idx < 20:
                speakers.add(s[:idx].strip())
    #================ helping Mayor De Blasio have a voice ===============#
    if deBlasio:
        
        speakers.remove("DE BLASIO")
        speakers.add("DEBLASIO")
    #================ helping Mayor De Blasio have a voice ===============#
    return speakers

In [ ]:
candidates1 = analyzeCandidates(night1, True)
for candidate in candidates1.keys():
    print(candidate, "made", candidates1[candidate].numOfContributions(), "contributions.")

In [ ]:
candidates1.pop('ANNOUNCER', None)
candidates1.pop('(UNKNOWN)', None);

In [ ]:
moderators = {'GUTHRIE', 'HOLT', 'MADDOW', 'TODD', 'DIAZ-BALART', "DIAZ BALART", "BALART"}
candidates = set()

In [ ]:
for speaker in list(candidates1):
    if speaker not in moderators:
        candidates.add(speaker)

In [ ]:
candidates2 = analyzeCandidates(night2)
for candidate in candidates2.keys():
    print(candidate, "made", candidates2[candidate].numOfContributions(), "contributions.")

In [ ]:
candidates2.pop('UNKNOWN', None);

In [ ]:
candidates = {}
candidates.update(candidates1)
candidates.update(candidates2)

In [ ]:
for speaker in moderators:
    candidates.pop(speaker, None)
len(candidates)

Great, now we have a mapping a Speaker object for each candidate. Now we can make a single method in the class fo the plots. So this object also simplifies our graphing and abstracts that code away into a method that every speaker has.

# July 30th and July 31st Debates